In [1]:
import os
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\\bin")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\libnvvp")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\extras\CUPTI\lib64")
os.add_dll_directory("C:\Program Files\\NVIDIA GPU Computing Toolkit\CUDA\\v10.1\include")
os.add_dll_directory("C:\Program Files\\NVIDIA Corporation\\Nsight Compute 2019.4.0")

<AddedDllDirectory('C:\\Program Files\\NVIDIA Corporation\\Nsight Compute 2019.4.0')>

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5848108313342657410,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 12327166193688964156
 physical_device_desc: "device: XLA_CPU device",
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 4817616896
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3908503974934489251
 physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5",
 name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9471200887679321275
 physical_device_desc: "device: XLA_GPU device"]

In [4]:
 img_spec = tf.TensorSpec((128,128,3) , dtype=tf.float32)
 feature_spec = tf.TensorSpec((1 , 39) , dtype=tf.float32)
 dataset = tf.data.experimental.load(
     'F:\\messy code\\afg ordered\\second generation',
     element_spec = (img_spec,feature_spec),
    compression='GZIP', 
     reader_func=None )

In [5]:
dataset

<_LoadDataset shapes: ((128, 128, 3), (1, 39)), types: (tf.float32, tf.float32)>

In [6]:
# face_path = "F:\\messy code\\afg ordered\second generation\\faces"
# dataset = tf.data.Dataset.list_files(str('{}\\{}'.format(face_path,'*\\*')))

In [7]:
# feature_dir = "F:\messy code\\afg ordered\second generation\getchu hopefully last\content\\features"
# import sys
# sys.path.append("F:\messy code\\afg ordered\second generation\data.py")
# import data
# def name_to_picture(path_tensor):
#     path = path_tensor.numpy().decode("utf-8")
#     image = tf.io.read_file(path)
#     image = tf.image.decode_jpeg(image)
#     image = tf.image.convert_image_dtype(image, tf.float32)
#     image = tf.image.resize(image, [128, 128])

#     name = data.path_to_name(path)
#     feature_path = data.name_to_feature_path(name , feature_dir)

#     feature_vector = np.load(feature_path)

#     feature_len = feature_vector.shape[0]

#     feature_vector = feature_vector.reshape(1, feature_len)
#     return image , feature_vector


# dataset = dataset.map(lambda x: tf.py_function(name_to_picture, [x], Tout = [tf.float32, tf.float32]))


In [8]:
tensor = next(iter(dataset))
image = tensor[0]
feature = tensor[1]
d_steps = 3
gp_weight = 3

image_shape = image.numpy().shape
feature_shape = feature.numpy().shape
noise_len = 128

print("image shape is : {}".format(image_shape))
print("feature shape is : {}".format(feature_shape))

image shape is : (128, 128, 3)
feature shape is : (1, 39)


In [9]:
batch_size = 64 
#dataset = dataset.batch(batch_size)

In [10]:
import sys 
sys.path.insert(0, 'F:\messy code\\afg ordered\second generation')
from generator import generator_model
generator_input_shape = (None , noise_len + feature_shape[1])
generator = generator_model(input_shape = generator_input_shape)


final Tensor("Sigmoid:0", shape=(None, 128, 128, 3), dtype=float32)


In [11]:
generator.summary()

Model: "generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 167)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 16384)        2752512     input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16384)        65536       dense[0][0]                      
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 16384)        0           batch_normalization[0][0]        
__________________________________________________________________________________________

In [12]:
from discriminator import discriminator_model
discriminator = discriminator_model()

In [13]:
discriminator.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 63, 63, 32)   1568        input_2[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 63, 63, 32)   0           conv2d_37[0][0]                  
__________________________________________________________________________________________________
conv2d_38 (Conv2D)              (None, 63, 63, 32)   9248        leaky_re_lu[0][0]                
_______________________________________________________________________________________

In [14]:
optimizer = tf.optimizers.Adam(beta_1=0.5)

In [15]:
import WGAN_GP
from discriminator import discriminator_loss 
from generator import generator_loss
wgan = WGAN_GP.WGAN(discriminator , generator)
wgan.compile( optimizer , optimizer , discriminator_loss , generator_loss)

In [16]:
wgan.fit(dataset, batch_size= 64, epochs=10)

Epoch 1/10
  187/32276 [..............................] - ETA: 1:15:59 - d_loss: nan - g_loss: nan

KeyboardInterrupt: 